In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
vocab = list(set(sentence))
print(vocab)

['Repeat', 'for', 'is', 'memory', 'best', 'medicine', 'the']


In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>'] = 0

In [5]:
print(word2index)

{'Repeat': 1, 'for': 2, 'is': 3, 'memory': 4, 'best': 5, 'medicine': 6, 'the': 7, '<unk>': 0}


In [6]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = dict(zip(word2index.values(), word2index.keys()))
print(index2word)

{1: 'Repeat', 2: 'for', 3: 'is', 4: 'memory', 5: 'best', 6: 'medicine', 7: 'the', 0: '<unk>'}


In [7]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [8]:
X, Y = build_data(sentence, word2index)

In [10]:
print(X)
print(Y)

tensor([[1, 3, 7, 5, 6, 2]])
tensor([[3, 7, 5, 6, 2, 4]])


In [12]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.
    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [13]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [14]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [15]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[-0.0257, -0.0410, -0.0233, -0.0191,  0.2299,  0.1283, -0.1495,  0.0993],
        [-0.1732,  0.0171,  0.0071, -0.0233,  0.3497, -0.0815, -0.1195,  0.1687],
        [-0.0330, -0.2707,  0.1174, -0.0889,  0.3861,  0.0993, -0.0211,  0.0308],
        [-0.1411, -0.1624,  0.2690, -0.1704,  0.4379,  0.2320, -0.0866,  0.1866],
        [-0.1252, -0.1828, -0.1061,  0.0672,  0.0307,  0.3649, -0.1766, -0.1793],
        [-0.2705, -0.2163,  0.1364,  0.0474,  0.2532,  0.0999, -0.0355,  0.1006]],
       grad_fn=<ViewBackward>)


In [16]:
print(output.shape)

torch.Size([6, 8])


In [17]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]


In [18]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0625 
Repeat memory memory memory memory best memory

[41/201] 1.5400 
Repeat the the best medicine for memory

[81/201] 0.9286 
Repeat the the best medicine for memory

[121/201] 0.4683 
Repeat is the best medicine for memory

[161/201] 0.2514 
Repeat is the best medicine for memory

[201/201] 0.1474 
Repeat is the best medicine for memory

